In [ ]:
df1['comprometimento_renda'] = df1[['divida_atual', 'renda_anual']].apply(lambda x: (x['divida_atual'] / x['renda_anual'])*100, axis=1)

df1['comprometimento_renda'] = df1['comprometimento_renda'].apply(lambda x:'Não' if x < 30 
                                                                  else 'Sim')


df1['num_pgtos_atrasados'] = df1['num_pgtos_atrasados'].apply(lambda x: 
              'Excelente' if x == 0 else 
              'Bom' if 1 <= x <= 10 else 
              'Regular' if 11 <= x <= 20 else 'Ruim' )

In [ ]:
df5 = df1.copy()

In [ ]:
# data preparation
#normalizacao
ss = StandardScaler()

# saldo_autal
df5['saldo_atual'] = ss.fit_transform( df5[['saldo_atual']].values )
pickle.dump( ss, open( 'parameter/saldo_atual_scaler.pkl', 'wb') )

# taxa_utilizacao_credito
df5['taxa_utilizacao_credito'] = ss.fit_transform( df5[['taxa_utilizacao_credito']].values )
pickle.dump( ss, open( 'parameter/taxa_utilizacao_credito_scaler.pkl', 'wb') )

In [ ]:
rescaling
rs = RobustScaler()
mms = MinMaxScaler()



## Robust Scaler
# idade
df5['idade'] = rs.fit_transform( df5[['idade']].values )
pickle.dump( rs, open( 'parameter/idade_scaler.pkl', 'wb') )


# renda_anual
df5['renda_anual'] = rs.fit_transform( df5[['renda_anual']].values )
pickle.dump( rs, open( 'parameter/renda_anual_scaler.pkl', 'wb') )

# valor_em_investimentos
df5['valor_em_investimentos'] = rs.fit_transform( df5[['valor_em_investimentos']].values )
pickle.dump( rs, open( 'parameter/valor_em_investimentos_scaler.pkl', 'wb') )

# num_emprestimos
df5['num_emprestimos'] = rs.fit_transform( df5[['num_emprestimos']].values )
pickle.dump( rs, open( 'parameter/num_emprestimos_scaler.pkl', 'wb') )

# num_contas_bancarias
df5['num_contas_bancarias'] = rs.fit_transform( df5[['num_contas_bancarias']].values )
pickle.dump( rs, open( 'parameter/num_contas_bancarias_scaler.pkl', 'wb') )

# num_cartoes_credito
df5['num_cartoes_credito'] = rs.fit_transform( df5[['num_cartoes_credito']].values )
pickle.dump( rs, open( 'parameter/num_cartoes_credito_scaler.pkl', 'wb') )

# # num_pgtos_atrasados
# df5['num_pgtos_atrasados'] = rs.fit_transform( df5[['num_pgtos_atrasados']].values )
# pickle.dump( rs, open( 'parameter/num_pgtos_atrasados_scaler.pkl', 'wb') )

# num_consultas_credito
df5['num_consultas_credito'] = rs.fit_transform( df5[['num_consultas_credito']].values )
pickle.dump( rs, open( 'parameter/num_consultas_credito_scaler.pkl', 'wb') )

# taxa_juros
df5['taxa_juros'] = rs.fit_transform( df5[['taxa_juros']].values )
pickle.dump( rs, open( 'parameter/taxa_juros_scaler.pkl', 'wb') )




In [ ]:
## MinMax Scaler
# dias_atraso_dt_venc
df5['dias_atraso_dt_venc'] = mms.fit_transform( df5[['dias_atraso_dt_venc']].values )
pickle.dump( rs, open( 'parameter/dias_atraso_dt_venc_scaler.pkl', 'wb') )

# divida_atual
df5['divida_atual'] = mms.fit_transform( df5[['divida_atual']].values )
pickle.dump( rs, open( 'parameter/divida_atual_scaler.pkl', 'wb') )

In [ ]:
# ENCODING
le = LabelEncoder()


df5['investe_exterior'] = le.fit_transform(df5['investe_exterior'])
df5['pessoa_polit_exp'] = le.fit_transform(df5['pessoa_polit_exp'])
df5['comprometimento_renda'] = le.fit_transform(df5['comprometimento_renda'])

lim_adi_dict = {'Negar': 0, 'Conceder': 1}
df5['limite_adicional'] = df5['limite_adicional'].map(lim_adi_dict)

pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
df5['num_pgtos_atrasados'] = df5['num_pgtos_atrasados'].map(pg_atrasados_dict)

In [ ]:
# Dividir dataset entre Treino (80%) e Teste (20%) 
X = df5.drop('limite_adicional', axis = 1).copy()
y = df5['limite_adicional'].copy()

x_train, x_test, y_train, y_test = ms.train_test_split(X, y, stratify = y, test_size = 0.20, random_state = 42)

In [ ]:
# training and test dataset for boruta
x_train_n = x_train.values
y_train_n = y_train.values.ravel()

# define RadomForestClassifier
rf = RandomForestClassifier( n_jobs= -1 ) # warm_start = True,

# define boruta
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( x_train_n, y_train_n )

In [ ]:
cols_selected = boruta.support_.tolist()

# best features
x_train_fs = x_train.copy()
cols_selected_boruta = x_train_fs.iloc[:, cols_selected].columns.to_list()


# not selected boruta
cols_not_selected_boruta = list(np.setdiff1d(x_train_fs.columns, cols_selected_boruta))

In [ ]:
cols_selected_boruta